In [1]:
import ee
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob
from datetime import date

from uuid import uuid4


from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
import seaborn as sns

import dask.dataframe as dd
import geopandas as gpd
from heatwave_1 import detect
from datetime import timedelta
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt

sns.set_style('darkgrid')

In [2]:
daily_max_dic = {}

for state in ['California', 'Oregon', 'Washington', 'British Columbia']:
            
    daily_max_files = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max*')
    
    temp = dd.read_csv(daily_max_files)
    temp = temp.compute()
    daily_max_dic[state] = temp
    

full_df = pd.concat(daily_max_dic.values())

full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

full_df['location'] = full_df['county'] + ', ' + full_df['state']
full_df.drop_duplicates(['location', 'datetime'], inplace=True)

full_df = full_df[(full_df['month'] != 10)]
full_df = full_df[(full_df['month'] != 4)]

full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

# Identify heatwaves

In [ ]:
#create dictionary containing each county's heatwave periods
heatwave_dic = {}
for county in full_df['location'].unique():
    county_df = full_df[(full_df['location'] == county)]
    county_df.reset_index(inplace=True)
    county_df.sort_values(by='datetime', inplace=True)
    county_df.drop_duplicates(['county', 'datetime'], inplace=True)
    
    #call 'detect' function from heatwave_1.py
    heatwave_dic[county] = detect(county_df['datetime_ord'].values, county_df['temperature_2m'].values, pctile=90, climatologyPeriod = [None, None], minDuration=2)


#create dictionary of dates for each heatwave
heatwave_dates = {}

#create dictionary of heatwave start day and lengths
consec_dates = {}

#generate unique ID code for each heatwave

heatwave_IDs = {}
for county in heatwave_dic.keys(): 
    consec_dates[county] = {}
    heatwave_dates[county] = []
    heatwave_IDs[county] = {}


    heatwave_starts = heatwave_dic[county][0]['date_start']
    heatwave_ends = heatwave_dic[county][0]['date_end']
    for i in range(len(heatwave_starts)):
        heatwave_code = str(uuid4())
        heatwave_days = pd.date_range(heatwave_starts[i],heatwave_ends[i],freq='d')
        length_heatwaves = len(heatwave_days)
        consec_dates[county][heatwave_days[0]] = length_heatwaves
        for item in heatwave_days:
            heatwave_dates[county].append(item)
            heatwave_IDs[county][item] = heatwave_code
            
            
heat_df = pd.DataFrame.from_dict(heatwave_dates, orient='index').unstack().reset_index()
heat_df.rename(columns={'level_1':'location', 0:'datetime'}, inplace=True)


heat_df.dropna(inplace=True)
heat_df['heatwave']=1

In [9]:
#merge full temperature dataset with identified 
full_df = full_df.merge(heat_df, on=['location', 'datetime'], how='left')

full_df['heatwave'] = full_df['heatwave'].replace(np.nan, 0)

In [10]:
full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [11]:
full_df[full_df['heatwave'] == 1].groupby('state')['temperature_2m'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
British Columbia,2787.0,26.377772,3.446535,17.107688,24.266970,26.536002,28.533001,40.565983
California,5325.0,35.858835,4.197082,24.487600,33.254892,35.651928,38.916122,48.243633
Oregon,3154.0,33.245785,2.930243,23.204296,31.778013,33.445842,35.045235,44.444990
Washington,3631.0,32.303933,4.454247,19.978627,29.185256,32.518465,35.240538,46.929940


In [12]:
#create dictionary of unique heatwave ID codes
codes = pd.DataFrame.from_dict(heatwave_IDs, orient='index').unstack().reset_index()

In [13]:
codes.columns=['datetime', 'location', 'heat_ID']

In [14]:
codes.dropna(inplace=True)

In [15]:
codes.head()

,datetime,location,heat_ID
0,2016-06-19,"San Bernardino, California",6cfb3653-2366-4594-9e24-805dddcc888e
1,2016-06-19,"Imperial, California",20571f04-bbb0-4000-aabf-709d84ae10dd
2,2016-06-19,"Los Angeles, California",646fe7cf-e15d-4d5e-a3a1-079d6df2409e
3,2016-06-19,"Orange, California",f82f809c-c265-4885-95d6-f31068e8d821
4,2016-06-19,"Riverside, California",8e22296c-8e3b-4a5c-a320-d6e58831660e


In [16]:
#get dataframe of heatwave durations
consec = pd.DataFrame.from_dict(consec_dates, orient='index').unstack().reset_index()

In [17]:
consec.columns= ['datetime', 'location', 'Heat Wave Duration']

In [18]:
consec['Heat Wave Duration'].max()

12.0

In [19]:
consec.head()

,datetime,location,Heat Wave Duration
0,2016-06-19,"San Bernardino, California",3.0
1,2016-06-19,"Imperial, California",2.0
2,2016-06-19,"Los Angeles, California",3.0
3,2016-06-19,"Orange, California",2.0
4,2016-06-19,"Riverside, California",2.0


In [20]:
consec.dropna(inplace=True)

In [21]:
full_df = full_df.merge(codes, on=['datetime', 'location'])

In [22]:
full_df = full_df[full_df['year'] >= 2020]



In [23]:
full_df = full_df.merge(consec, on=['datetime', 'location'], how='left')

In [24]:
heatwaves = full_df.copy()

In [25]:
heatwaves['location'] = heatwaves['county'] + ', ' + heatwaves['state']

In [26]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [27]:
heatwaves = heatwaves.merge(consec, on=['datetime', 'location'], how='left')

In [28]:
heatwaves = heatwaves[heatwaves['month'] != 10]

In [29]:
heatwaves.sort_values(by='datetime', inplace=True)

In [30]:
heatwaves.groupby('state')['temperature_2m'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
British Columbia,1336.0,27.005151,3.625918,17.107688,24.712823,26.909636,29.143250,40.565983
California,2762.0,36.237737,4.209442,24.487600,33.552885,36.214873,39.277454,47.254585
Oregon,1644.0,33.544126,3.093504,23.544898,32.059122,33.624946,35.326932,44.444990
Washington,1782.0,33.006739,4.727335,19.978627,29.656325,33.284610,36.300178,46.929940


In [31]:
#save file
heatwaves.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwave_clean_90_v1.csv')